In [1]:
import sys
print(sys.version)

3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]


In [3]:
import pandas as pd
import datetime
import numpy as np

In [4]:
df_patient=pd.read_csv('C:/Users/lg/Desktop/COVID_19/PatientInfo.csv', dtype={'patient_id':str, 'infected_by':str})

In [10]:
# 지역 이름 수정
def fix_region(row):
    fixdic = {
        ('경기도','구리'):('경기도','구리시'),
        ('경기도','수원'):('경기도','수원시'),
        ('경기도','양평시'):('경기도','양평'),
        ('대구','경산시'):('경상북도','경산시'),
        ('대구','달성군'):('대구','달성'),
        ('대구','상곡 동'):('대구','대구'),
        ('대구','영천시'):('경상북도','영천시'),
        ('대구','이천동'):('대구','남구'),
        ('대전','세종'):('세종','세종'),
        ('전라남도','경산시'):('경상북도','경산시'),
        ('경상남도','하만'):('경상남도','함안군'),
        ('서울','동구 구'):('서울','동작구'),
        ('인천','미추홀 구'):('인천','미추홀구'),
        ('전라북도','지난'):('전라북도','진안군'),
        ('전라남도','시난'):('전라남도','신안군')}
    if (row.province, row.city) in fixdic.keys():
        row.province, row.city = fixdic[(row.province, row.city)]
    if (row.province == row.city) or (row.city=='기타'):
        row.city = np.nan
    return row
df_patient = df_patient.apply(fix_region, axis=1)

In [5]:
#확진자의 확진일 결측치 
df_patient = df_patient.query('confirmed_date==confirmed_date')

In [7]:
df_patient['confirmed_dt']=df_patient['confirmed_date'].apply(lambda x:datetime.datetime.fromisoformat(x))

In [11]:
min_date=df_patient['confirmed_dt'].min()
max_date=df_patient['confirmed_dt'].max()

In [12]:
date_range=list(pd.date_range(start=min_date, end=max_date))

In [14]:
df_dummy_date=pd.DataFrame.from_dict({'patiend_id':[-1]*len(date_range),\
                                     'province':['dummy_prov']*len(date_range),\
                                     'city':['dummy_city']*len(date_range),
                                     'confirmed_dt':date_range})

In [15]:
df_patient=df_patient.append(df_dummy_date)

In [16]:
df_patient['valid']=df_patient['patient_id'].apply(lambda x: 1 if isinstance(x,str) else 0)

In [17]:
df_patient[]

,patient_id,sex,age,country,province,city,infection_case,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state,confirmed_dt,patiend_id,valid
0,1000000001,male,50s,Korea,서울,강서구,overseas inflow,NaN,75,2020-01-22,2020-01-23,2020-02-05,NaN,released,2020-01-23,NaN,1
1,1000000002,male,30s,Korea,서울,중랑구,overseas inflow,NaN,31,NaN,2020-01-30,2020-03-02,NaN,released,2020-01-30,NaN,1
2,1000000003,male,50s,Korea,서울,종로구,contact with patient,2002000001,17,NaN,2020-01-30,2020-02-19,NaN,released,2020-01-30,NaN,1
3,1000000004,male,20s,Korea,서울,마포구,overseas inflow,NaN,9,2020-01-26,2020-01-30,2020-02-15,NaN,released,2020-01-30,NaN,1
4,1000000005,female,20s,Korea,서울,성북구,contact with patient,1000000002,2,NaN,2020-01-31,2020-02-24,NaN,released,2020-01-31,NaN,1


In [18]:
df_patient.to_csv('C:/Users/lg/Desktop/COVID_19/patientinfo.csv')